In [5]:
using CSV;
using LowRankApprox;
include("../code/julia/ash.jl");
include("../code/julia/mixSQP_time.jl");

In [6]:
include("../code/julia/makedata.jl");
L = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));

In [51]:
QPsubprob(L, initialize = "sparse", warmstart = false)

Dict{String,Any} with 5 entries:
  "timing"      => [0.000855983, 0.0010097, 0.000930596, 0.00099004, 0.00105497…
  "x"           => [0.496888, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "linesearch"  => [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1…
  "totalQPtime" => 0.0170546
  "numiter"     => 23

In [52]:
QPsubprob(L, initialize = "sparse", warmstart = true)

Dict{String,Any} with 5 entries:
  "timing"      => [0.00068577, 0.000881309, 0.000975159, 0.000894507, 0.000886…
  "x"           => [0.496888, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "linesearch"  => [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1…
  "totalQPtime" => 0.0157574
  "numiter"     => 23

In [53]:
srand(1)
x = [randn(10^6);3*randn(10^6)];
s = ones(2*10^6); 
L = ash(x,s, mult = 1.05)[3];

In [54]:
QPsubprob(L, initialize = "sparse", warmstart = false)

Dict{String,Any} with 5 entries:
  "timing"      => [0.00666852, 0.000909547, 0.00124374, 0.000965151, 0.0008556…
  "x"           => [0.499933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "linesearch"  => [7.0, 5.0, 4.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,…
  "totalQPtime" => 0.0182387
  "numiter"     => 16

In [55]:
QPsubprob(L, initialize = "sparse", warmstart = true)

Dict{String,Any} with 5 entries:
  "timing"      => [0.00691269, 0.000331555, 0.000515658, 0.000268786, 0.000453…
  "x"           => [0.499933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0…
  "linesearch"  => [7.0, 5.0, 4.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,…
  "totalQPtime" => 0.0152202
  "numiter"     => 16